<a href="https://colab.research.google.com/github/hibameo/OPENAI_SDK_OPENROUTER/blob/main/OPENAI_SDK_OPENROUTER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 971.8 kB/s eta 0:00:00


In [2]:
import nest_asyncio
nest_asyncio.apply()

## Provider Config

In [3]:
from google.colab import userdata

OPENROUTER_API_KEY = userdata.get("OPENROUTER_API_KEY")

In [4]:
#Reference: https://openrouter.ai/docs/quickstart

BASE_URL = "https://openrouter.ai/api/v1"
MODEL = "mistralai/mistral-small-24b-instruct-2501:free"

# Some other free models on 26th March:
# https://openrouter.ai/deepseek/deepseek-chat-v3-0324:free
# https://openrouter.ai/google/gemini-2.5-pro-exp-03-25:free

## 1. Using the OpenRouter API directly

In [5]:
BASE_URL

'https://openrouter.ai/api/v1'

In [10]:
import requests
import json

response = requests.post(
  url=f"https://openrouter.ai/api/v1/chat/completions",
  headers={
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
  },
  data=json.dumps({
    "model": MODEL,
    "messages": [
      {
        "role": "user",
        "content": "Hi, i have 1000 pkr i want you to convert into usd"
      }
    ]
  })
)

print(response.json())

{'id': 'gen-1749802855-B1vvi7Y1AaxzSYyXbyG4', 'provider': 'Chutes', 'model': 'mistralai/mistral-small-24b-instruct-2501:free', 'object': 'chat.completion', 'created': 1749802855, 'choices': [{'logprobs': None, 'finish_reason': 'stop', 'native_finish_reason': 'stop', 'index': 0, 'message': {'role': 'assistant', 'content': "To convert 1000 Pakistani Rupees (PKR) to US Dollars (USD), you need to know the current exchange rate. Exchange rates fluctuate daily, so I can't provide the exact rate without knowing the current market value.", 'refusal': None, 'reasoning': None}}], 'usage': {'prompt_tokens': 23, 'completion_tokens': 52, 'total_tokens': 75, 'prompt_tokens_details': None}}


In [11]:
data = response.json()
data['choices'][0]['message']['content']

"To convert 1000 Pakistani Rupees (PKR) to US Dollars (USD), you need to know the current exchange rate. Exchange rates fluctuate daily, so I can't provide the exact rate without knowing the current market value."

## 2. Using OpenAI Agents SDK

In [14]:
import asyncio
from openai import AsyncOpenAI # chat completions
from agents import Agent, OpenAIChatCompletionsModel, Runner, set_tracing_disabled

set_tracing_disabled(disabled=True) # Open AI Tracing == Disable

client = AsyncOpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url=BASE_URL
)


async def main():
    # This agent will use the custom LLM provider
    agent = Agent(
        name="LahoreTA",
        instructions="You only respond in english.",
        model=OpenAIChatCompletionsModel(model=MODEL, openai_client=client),
    )

    result = await Runner.run(
        agent, # starting agent
        "What is your name?.", # request
    )
    print(result.final_output)


asyncio.run(main())

I don't have a name. I'm a text-based AI model and I don't have personal experiences, feelings, or a name. I'm here to provide helpful, respectful, and honest assistance to the best of my ability. If you'd like to give me a name to use during our conversation, you're welcome to do so! What would you like to call me?


In [15]:
response.json()

{'id': 'gen-1749802855-B1vvi7Y1AaxzSYyXbyG4',
 'provider': 'Chutes',
 'model': 'mistralai/mistral-small-24b-instruct-2501:free',
 'object': 'chat.completion',
 'created': 1749802855,
 'choices': [{'logprobs': None,
   'finish_reason': 'stop',
   'native_finish_reason': 'stop',
   'index': 0,
   'message': {'role': 'assistant',
    'content': "To convert 1000 Pakistani Rupees (PKR) to US Dollars (USD), you need to know the current exchange rate. Exchange rates fluctuate daily, so I can't provide the exact rate without knowing the current market value.",
    'refusal': None,
    'reasoning': None}}],
 'usage': {'prompt_tokens': 23,
  'completion_tokens': 52,
  'total_tokens': 75,
  'prompt_tokens_details': None}}